# Importações

In [26]:
import pandas as pd
import os
from datetime import datetime
import time

# Tabelas

In [27]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

caminho_usuarios = os.path.join(base_dir, 'Bases', 'Tabela-Usuario.csv')
caminho_perfil = os.path.join(base_dir, 'Bases', 'Tabela-Perfil.csv')
caminho_pessoa = os.path.join(base_dir, 'Bases', 'Tabela-Pessoa.csv')
caminho_sessao = os.path.join(base_dir, 'Bases', 'Tabela-Sessao.csv')
caminho_equipes = os.path.join(base_dir, 'Bases', 'Tabela-Equipes.csv')
caminho_recompensas = os.path.join(base_dir, 'Bases', 'Tabela-Recompensa.csv')
caminho_prestigio = os.path.join(base_dir, 'Bases', 'Tabela-Prestigio.csv')

tabela_usuario = pd.read_csv(caminho_usuarios, sep=';')
tabela_perfil = pd.read_csv(caminho_perfil, sep=';')
tabela_pessoa = pd.read_csv(caminho_pessoa, sep=';')
tabela_sessao = pd.read_csv(caminho_sessao, sep=';')
tabela_equipes = pd.read_csv(caminho_equipes, sep=';')
tabela_recompensas = pd.read_csv(caminho_recompensas, sep=';')
tabela_prestigio = pd.read_csv(caminho_prestigio, sep=';', encoding='latin1')

### VAR

In [28]:
novo_id_usuario = len(tabela_usuario) +  1
novo_id_sessao = len(tabela_sessao) + 1

# Defs

In [29]:
def cadastro():
    nome_user = input("Digite seu nome de usuário: ")
    nome = input("Digite seu nome: ")
    email = input("Digite seu email: ")
    data_nasc = input("Digite sua data de nascimento: ") 
    senha = input("Digite sua senha: ")
    telefone = input("Digite seu telefone: ")
    estado = input("Digite a sigla do seu estado: ")
    pais = input("Digite seu país: ")
    return nome_user, nome, email, senha, data_nasc, telefone, estado, pais

def login(infos, id):

    while True:
        email = input("Digite seu email: ")
        senha = input("Digite sua senha: ")
        listaEmail = list(infos['Email'])
        listaSenha = list(infos['Senha'])
        if email in listaEmail:
            indice = listaEmail.index(email)
            if senha == str(listaSenha[indice]): 
                print("Login realizado com sucesso!")
                infosSessao = {
                    "Id. Sessao":id,
                    "Id. Usuario":infos['Id. Usuario'][indice],
                    "Data Login":datetime.now(),
                    "Data Logout":None
                }
                return infosSessao
            else:
                print(f"Senha ou Email incorreto!{listaSenha[indice]} e {senha}")
                continue
        else:
            while True:
                tentativa = int(input("Senha incorreta, digite 1 para tentar novamente ou 2 para sair: "))
                if tentativa == 1:
                    break
                elif tentativa == 2:
                    return None
                else:
                    print("Opção inválida!")

def perfilizacao(equipe):
    
    equipes = list(equipe['Equipe'])
    print("Temos essas opções de equipe: ")
    for i, e in enumerate(equipes,1):
        print(f'{i}. {e}')
    while True:
        escolha = input("Digite o numero da sua escolha: ")

        if escolha == '1':
            print("Você escolheu a equipe", equipes[0], "Seu perfil está completo")
            return equipes[0]
        elif escolha == '2':
            print("Você escolheu a equipe", equipes[1], "Seu perfil está completo")
            return equipes[1]
        elif escolha == '3':
            print("Você prefiriu não ter uma equipe, seu perfil está completo")
            return equipes[2]
        else:
            print("Opção inválida!")

def verificar_prestigio(tabela_perfil, tabela_prestigio):
    perfis_list = tabela_perfil.to_dict(orient='records')
    prestigios_list = tabela_prestigio.to_dict(orient='records')
    
    for perfil in perfis_list:
        total_pontos = perfil["Total Pontos"]
       
        prestigio_qualificado = None
        for prestigio in prestigios_list:
            if prestigio["Ponto Min"] <= total_pontos:
                if prestigio_qualificado is None or prestigio["Ponto Min"] > prestigio_qualificado["Ponto Min"]:
                    prestigio_qualificado = prestigio
        
        if prestigio_qualificado:
            perfil["Id. Prestigio"] = prestigio_qualificado["Id. Prestigio"]
    
    perfis_atualizado_df = pd.DataFrame(perfis_list)
    
    return perfis_atualizado_df

def verificar_recompensa(perfis, recompensas):
    perfis_list = perfis.to_dict(orient='records')
    recompensas_list = recompensas.to_dict(orient='records')

    
    for perfil in perfis_list:
        for recompensa in recompensas_list:
            status = recompensa.get("Status", "").strip()
            if status != "Ativo":
                continue 
            if perfil["Id. Prestigio"] >= recompensa["Id. Prestigio"]:
                if recompensa["Id. Equipe"] is None or perfil["Id. Equipe"] == recompensa["Id. Equipe"]:
                    if recompensa["metodo"] == "merito":
                       if recompensa["Num pessoas"] < recompensa["Limite Pessoas"]:
                            print(f"{perfil['Nome']} ganhou a recompensa: {recompensa['Recompensa']}")
                            recompensa["Num pessoas"] += 1
                            adicionar_recompensa_usuario(perfil, recompensa["Id. Recompensa"], perfis_list)
                            if recompensa["Num pessoas"] == recompensa["Limite Pessoas"]:
                                recompensa["Status"] = "Inativo"
                                print(f"A recompensa {recompensa['Recompensa']} atingiu o limite e está agora inativa.")
                    elif recompensa["metodo"] == "sorteio":
                        print(f"{perfil['Nome']} está concorrendo ao sorteio da recompensa: {recompensa['Recompensa']}")
            

    perfis_atualizado_df = pd.DataFrame(perfis_list)
    recompensas_atualizado_df = pd.DataFrame(recompensas_list)
    
    return perfis_atualizado_df, recompensas_atualizado_df
    
def adicionar_recompensa_usuario(perfil, id_recompensa, perfis_list):
    if perfil["Id. Recompensa"] == id_recompensa:
        print(f"{perfil['Nome']} já ganhou a recompensa {id_recompensa} e não pode ganhá-la novamente.")
        return  
    if perfil["Id. Recompensa"] == 0:
        perfil["Id. Recompensa"] = id_recompensa
        print(f"Recompensa {id_recompensa} adicionada ao perfil de {perfil['Nome']}")
    else:
        novo_perfil = perfil.copy()
        novo_perfil["Id. Recompensa"] = id_recompensa
        perfis_list.append(novo_perfil)
        print(f"Recompensa {id_recompensa} adicionada ao perfil de {perfil['Nome']}")

def programa(tabela_usuario, tabela_perfil, tabela_equipes, tabela_pessoa, tabela_recompensas, tabela_sessao):
    print("Bem-vindo ao Sistema de Fórmula E!")
    
    escolha = input("Digite 1 para Cadastro ou 2 para Login: ")
    
    if escolha == '1':
        nome_user, nome, email, senha, data_nasc, telefone, estado, pais = cadastro()
        
        novo_usuario = {
            "Id. Usuario": novo_id_usuario,
            "Nome Usuario": nome_user,
            "Email": email,
            "Senha": senha
        }
        novo_perfil = {
            "Id. Usuario": novo_id_usuario,
            "Nome Usuario": nome_user,
            "Nome": nome,
            "Id. Equipe": 0,
            "Id. Membro": 0,
            "Id. Prestigio": 0,
            "Id. Nivel": 0,
            "Id. Recompensa": 0,
            "Total Pontos": 0,
            "Num Sessoes": 0
        }
        nova_pessoa = {
            "Id. Usuario":novo_id_usuario,
            "Nome":nome,
            "Email":email,
            "Senha":senha,
            "Tel":telefone,
            "Data Nasc":data_nasc,
            "UF":estado,
            "Pais":pais
        }
        
        tabela_usuario = pd.concat([tabela_usuario, pd.DataFrame([novo_usuario])], ignore_index=True)
        tabela_perfil = pd.concat([tabela_perfil, pd.DataFrame([novo_perfil])], ignore_index=True)
        tabela_pessoa = pd.concat([tabela_pessoa, pd.DataFrame([nova_pessoa])], ignore_index=True)
        
        print(f"Usuário {nome_user} cadastrado com sucesso!")
        
        escolhaLogin = int(input("Digite 1 para fazer login e 2 para sair: "))
        if escolhaLogin == 1:
            infosSessao = login(tabela_usuario, novo_id_sessao)
            if infosSessao is None:
                print("Login cancelado!")
                return
        
            tabela_sessao = pd.concat([tabela_sessao, pd.DataFrame([infosSessao])], ignore_index=True)
            print("Sessão iniciada.")
        else:
            print("Obrigado por usar o sistema, nos vemos novamente!")

    elif escolha == '2':
        infosSessao = login(tabela_usuario, novo_id_sessao)
        if infosSessao is None:
            print("Login cancelado!")
            return
        
        tabela_sessao = pd.concat([tabela_sessao, pd.DataFrame([infosSessao])], ignore_index=True)
        print("Sessão iniciada.")
    
    else:
        print("Opção inválida.")
        return

    usuario_logado = tabela_perfil[tabela_perfil["Id. Usuario"] == infosSessao["Id. Usuario"]].iloc[0]
    
    if usuario_logado["Id. Equipe"] == 0:
        equipe_escolhida = perfilizacao(tabela_equipes)
        tabela_perfil.loc[tabela_perfil["Id. Usuario"] == usuario_logado["Id. Usuario"], "Id. Equipe"] = tabela_equipes[tabela_equipes["Equipe"] == equipe_escolhida]["Id. Equipe"].values[0]
    
    tabela_perfil = verificar_prestigio(tabela_perfil, tabela_prestigio)
    tabela_perfil, tabela_recompensas = verificar_recompensa(tabela_perfil, tabela_recompensas)
    
    encerrar = input("Deseja encerrar a sessão? Digite 'sim' ou 'não': ")
    
    if encerrar.lower() == 'sim':
        tempo_logout = datetime.now()
        tabela_sessao.loc[tabela_sessao["Id. Sessao"] == infosSessao["Id. Sessao"], "Data Logout"] = tempo_logout
        print("Sessão encerrada com sucesso!")
    else:
        tempo_logout = datetime.now()
        tabela_sessao.loc[tabela_sessao["Id. Sessao"] == infosSessao["Id. Sessao"], "Data Logout"] = tempo_logout
        print("Estamos trabalhando em novas funcionalidades. Sessão encerrada.")
    
    return tabela_usuario, tabela_perfil, tabela_sessao, tabela_recompensas, tabela_pessoa




# Funcionamento

In [31]:
Ligar = input("Você quer dar inpicio ao programa? S/N").lower()
if Ligar == "s":
    print("Iniciando...")
    tabela_usuario, tabela_perfil, tabela_sessao, tabela_recompensas, tabela_pessoa = programa(tabela_usuario, tabela_perfil, tabela_equipes, tabela_pessoa, tabela_recompensas, tabela_sessao)
    tabela_sessao.to_csv(caminho_sessao, sep=';', index=False)
    tabela_pessoa.to_csv(caminho_pessoa, sep=';', index=False)
    tabela_recompensas.to_csv(caminho_recompensas, sep=';', index=False)
    tabela_usuario.to_csv(caminho_usuarios, sep=';', index=False)
    tabela_perfil.to_csv(caminho_perfil, sep=';', index=False)
else:
    print("Programa encerrado.")


Iniciando...
Bem-vindo ao Sistema de Fórmula E!
Usuário Pcxx cadastrado com sucesso!
Login realizado com sucesso!
Sessão iniciada.
Temos essas opções de equipe: 
1. Mahindra
2. Porsche
3. Todas
Você escolheu a equipe Mahindra Seu perfil está completo
Fabiano Zague ganhou a recompensa: Porsche Fan
Fabiano Zague já ganhou a recompensa 1 e não pode ganhá-la novamente.
Sessão encerrada com sucesso!
